In [18]:
import pandas as pd
import numpy as np
from numpy import delete
import scipy.stats as stats
import time

Qué hay en este fichero:
- 1) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables sin suavizar
- 1.2) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables suavizadas
- 2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables sin suavizar
- 2.2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables suavizadas


# 0) Obtener los datos cada 15 mins de policromos:
Necesario para obtener coef. de correl que usen datos de policromos

In [2]:
def adaptar_len_policromos(FicheroCruce, FicheroExterior, FicheroHall, FicheroPolicromos, mesAño):
    """
    Atributos:
    1) FicheroCruce: Excel con datos del cruce
    2) FicheroExterior: Excel con datos del exterior
    3) FicheroHall: Excel con datos del hall
    4) FicheroPolicromos: Excel con datos de policromos
    5) mesAño: String que indique el mes y año al que pertenecen los datos 
    
    Devuelve:
    1) datosPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado; Una lista de listas, donde cada elemento (Tp, Tp_suavizada, ect.)
    contiene los datos cada 15 minutos de policromos de la variable correspondiente.
    
    Cosas a tener en cuenta:
    1) No se tiene en cuenta una diferencia temporal entre los datos iniciales de datospolicromos con Ficheroref porque 
    policromos_cada15min ya lo hace. (datospolicromos se obtiene a partir de llamar a policromos_cada15min para un fichero de 
    policromos)
    2) Pinta un mensaje de aviso en caso de que la longitud de policromos sea diferente a la del resto de ficheros 
    """
#     print('---------------------------------------------------------------------------------------------------------------')
#     print('---------------------------------------------------------------------------------------------------------------')
#     print(mesAño)
    #Leer ficheros
    Cruce = pd.read_excel(FicheroCruce)
    Exterior = pd.read_excel(FicheroExterior)
    FicheroHall = pd.read_excel(FicheroHall)
    Policromos = pd.read_excel(FicheroPolicromos)
    
    #Copiar los datos cada 15 mins---------------------------------------------------------------------------------
    a=0
    b=len(Policromos['Fecha/Tiempo'])
    min1 =FicheroHall['Fecha/Tiempo'][0].minute #Minutos del 1er dato del FAdicional
    Tp=[]#El 0 es un número entero, coge desde el 1er dato útil
    Tp_suavizada=[]
    CO2=[]
    CO2_suavizado=[]
    Fecha_Tiempo = []
    while a<b-1:
        division=(Policromos['Fecha/Tiempo'][a].minute-min1)/15
        division_entera=(Policromos['Fecha/Tiempo'][a].minute-min1)//15
        if  division == division_entera and division>=0:
#             print(Policromos['Temperatura C (C)'][a], Policromos['Tp_suavizada'][a], Policromos['CO2 C (ppm)'][a], 
#                   Policromos['CO2p_suavizado'][a], Policromos['Fecha/Tiempo'][a], 'dato a copiar')
            Tp.append(Policromos['Temperatura C (C)'][a])
            Tp_suavizada.append(Policromos['Tp_suavizada'][a])
            CO2.append(Policromos['CO2 C (ppm)'][a])
            CO2_suavizado.append(Policromos['CO2p_suavizado'][a])
            Fecha_Tiempo.append(Policromos['Fecha/Tiempo'][a])
#             print(Tp[len(Tp)-1], Tp_suavizada[len(Tp_suavizada)-1], CO2[len(CO2)-1], CO2_suavizado[len(CO2_suavizado)-1],
#                   Fecha_Tiempo[len(Fecha_Tiempo)-1], 'dato copiado')
        a=a+1
    # Tenemos listas de Tp, Tp_suavizada, CO2, CO2_suavizado y Fecha/Tiempo cada 15 mins 
    
    #recortar ficheros-----------------------------------------------------------------------------------------------------
    if len(FicheroHall['Fecha/Tiempo'])< len(Tp):
        Tp=Tp[:len(FicheroHall['Fecha/Tiempo'])]
        Tp_suavizada=Tp_suavizada[:len(FicheroHall['Fecha/Tiempo'])]
        CO2=CO2[:len(FicheroHall['Fecha/Tiempo'])]
        CO2_suavizado=CO2_suavizado[:len(FicheroHall['Fecha/Tiempo'])]
        Fecha_Tiempo=Fecha_Tiempo[:len(FicheroHall['Fecha/Tiempo'])]
#      int('Policromos es más corto-> Solucionar')   
    #pintar datos-----------------------------------------------------------------------------------------------------------
#     print('Datos finales-------------------')
#     print('lenCruce:', 'lenExterior:', 'lenHall:', 'lenPolicromos:')
#     print(len(Cruce['Fecha/Tiempo']), len(Exterior['Fecha/Tiempo']), len(FicheroHall['Fecha/Tiempo']), len(Tp))
#     print('--------------------------------')
#     print('1er dato Cruce:', '1er dato Exterior:', '1er dato hall:', '1er dato Policromos:')
#     print(Cruce['Fecha/Tiempo'][0], Exterior['Fecha/Tiempo'][0],FicheroHall['Fecha/Tiempo'][0],Fecha_Tiempo[0])
#     print('--------------------------------')
#     print('Último dato Cruce:', 'Último dato Exterior:', 'Último dato hall:', 'Último dato Policromos:')
#     print(Cruce['Fecha/Tiempo'][len(Cruce['Fecha/Tiempo'])-1], Exterior['Fecha/Tiempo'][len(Exterior['Fecha/Tiempo'])-1],
#           FicheroHall['Fecha/Tiempo'][len(FicheroHall['Fecha/Tiempo'])-1],Fecha_Tiempo[len(Tp)-1])
    return(Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha_Tiempo)

In [21]:
#Datos cada 15 mins en policromos:
# 2020
DatosPolicromos_10_20 = adaptar_len_policromos("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              "./altamira/2020/10.OCTUBRE/ZONA POLICROMOS.xlsx",'Octubre 2020')

DatosPolicromos_11_20 = adaptar_len_policromos("./altamira/2020/11.NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA POLICROMOS C.xlsx", 'Noviembre 2020')
DatosPolicromos_12_20 = adaptar_len_policromos("./altamira/2020/12.DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA POLICROMOS C.xlsx", 'Diciembre 2020')
#2021
DatosPolicromos_1_21 = adaptar_len_policromos("./altamira/2021/1.ENERO/ZONA CRUCE ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA EXTERIOR ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA ENTRADA ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA POLICROMOS ENERO.xlsx",'Enero 2021')
DatosPolicromos_2_21 = adaptar_len_policromos("./altamira/2021/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA POLICROMOS C.xlsx", 'Febrero 2021')
DatosPolicromos_3_21 = adaptar_len_policromos("./altamira/2021/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA POLICROMOS C.xlsx", 'Marzo 2021')
DatosPolicromos_4_21 = adaptar_len_policromos("./altamira/2021/4.ABRIL/ZONA CRUCE.xlsx",
                                                          "./altamira/2021/4.ABRIL/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/4.ABRIL/ZONA ENTRADA.xlsx",
                                                          "./altamira/2021/4.ABRIL/ZONA POLICROMOS C.xlsx", 'Abril 2021')
DatosPolicromos_5_21 = adaptar_len_policromos("./altamira/2021/5.MAYO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA POLICROMOS C.xlsx", 'Mayo 2021')
DatosPolicromos_6_21 = adaptar_len_policromos("./altamira/2021/6.JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/6.JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/6.JUNIO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/6.JUNIO/ZONA POLICROMOS C.xlsx", 'Junio 2021')
DatosPolicromos_7_21 = adaptar_len_policromos("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/7.JULIO/ZONA POLICROMOS C.xlsx", 'Julio 2021')
DatosPolicromos_8_21 = adaptar_len_policromos("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/8.AGOSTO/ZONA POLICROMOS C.xlsx", 'Agosto 2021')
DatosPolicromos_9_21 = adaptar_len_policromos("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA POLICROMOS C.xlsx", 'Septiembre 2021')
DatosPolicromos_10_21 = adaptar_len_policromos("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA POLICROMOS.xlsx", 'Octubre 2021')
DatosPolicromos_11_21 = adaptar_len_policromos("./altamira/2021/11. NOVIEMBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA POLICROMOS.xlsx", 'Noviembre 2021')
DatosPolicromos_12_21 = adaptar_len_policromos("./altamira/2021/12. DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2021/12. DICIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/12. DICIEMBRE/ZONA ENTRADA.xlsx",
                                                              "./altamira/2021/12. DICIEMBRE/ZONA POLICROMOS.xlsx", 'Diciembre 2021')
#2022
DatosPolicromos_1_22 = adaptar_len_policromos("./altamira/2022/1.ENERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/1.ENERO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/1.ENERO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/1.ENERO/ZONA POLICROMOS C.xlsx", 'Enero 2022')
DatosPolicromos_2_22 = adaptar_len_policromos("./altamira/2022/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA POLICROMOS C.xlsx", 'Febrero 2022')
DatosPolicromos_3_22 = adaptar_len_policromos("./altamira/2022/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/3.MARZO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/3.MARZO/ZONA POLICROMOS C.xlsx", 'Marzo 2022')
DatosPolicromos_4_22 = adaptar_len_policromos("./altamira/2022/4.ABRIL/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA POLICROMOS C.xlsx", 'Abril 2022')
#mayo 22 cruce vacío- A posteriori me hizo falta suavizar exterior
#Para poder utilizar la función suavizar poner otro archivo de cruce cualquiera.
DatosPolicromos_5_22 = adaptar_len_policromos("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA ENTRADA.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA POLICROMOS.xlsx", 'Mayo 2022')
DatosPolicromos_6_22 = adaptar_len_policromos("./altamira/2022/6. JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/6. JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/6. JUNIO/ZONA ENTRADA.xlsx",
                                                          "./altamira/2022/6. JUNIO/ZONA POLICROMOS C.xlsx", 'Junio 2022')
DatosPolicromos_7_22 = adaptar_len_policromos("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
                                                          "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/7.JULIO/ZONA POLICROMOS C.xlsx", 'Julio 2022')
DatosPolicromos_8_22 = adaptar_len_policromos("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/8. AGOSTO/ZONA POLICROMOS C.xlsx", 'Agosto 2022')
DatosPolicromos_9_22 = adaptar_len_policromos("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA POLICROMOS C.xlsx", 'Septiembre 2022')
DatosPolicromos_10_22 = adaptar_len_policromos("./altamira/2022/10. OCTUBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2022/10. OCTUBRE/ZONA POLICROMOS C.xlsx", 'Octubre 2022')
DatosPolicromos_11_22 = adaptar_len_policromos("./altamira/2022/11. NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA POLICROMOS C.xlsx", 'Noviembre 2022')
DatosPolicromos_12_22 = adaptar_len_policromos("./altamira/2022/12. DICIEMBRE/ZONA CRUCE D.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA POLICROMOS.xlsx", 'Diciembre 2022')

# 1) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables sin suavizar

In [4]:
def CoefPearsonCO2(NombreFicheroCruce, NombreFicheroExterior, NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
    Cruce= pd.read_excel(NombreFicheroCruce)
    Exterior = pd.read_excel(NombreFicheroExterior)
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce:
    CO2Cruce = []
    CO2Hall = []
    TCruce = []
    TExterior = []
    THall = []
    while a<len(listasPolicromos[2]):
        CO2Cruce.append(Cruce['CO2 D (ppm)'][a])
        CO2Hall.append(Hall['CO2 A (ppm)'][a])
        TCruce.append(Cruce["Temperatura D (° C)"][a])
        TExterior.append(Exterior["Tr Exterior (C Tr)"][a])
        THall.append(Hall['Temperatura A (° C)'][a])
        a=a+1
    CO2Policromos = listasPolicromos[2]
    TPolicromos = listasPolicromos[0]
    #Eliminar los nans y infs de las listas
#     print(len(TsPolicromos), len(TsExterior), len(TsHall))
    index=len(TPolicromos)-1
    while index>-1:
#         print(TsExterior[index], TsHall[index], TsPolicromos[index], index)
        if  sum([pd.isna(CO2Cruce[index]), pd.isna(CO2Hall[index]), pd.isna(TCruce[index]), pd.isna(TExterior[index]), 
                 pd.isna(THall[index]), pd.isna(listasPolicromos[0][index]), pd.isna(listasPolicromos[2][index])]) > 0:
#             print('borrar', TsExterior[index], TsHall[index], TsPolicromos[index], index)
            CO2Cruce.pop(index)
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
            TCruce.pop(index)
            TExterior.pop(index)
            THall.pop(index)
            TPolicromos.pop(index)
        index=index-1
#     print(len(TsPolicromos), TsExterior)
    #Diferencia de temeperaturas entre las salas y el exterior:
    #No deja hacer la resta directamente, hacerlo elemento por elemento en un bucle
    i=0
    Tc_menosTe= []
    Th_menosTe= []
    Tp_menosTe= []
#     print(len(TCruce), len(TExterior),len(THall),len(TPolicromos))
    while i<len(CO2Cruce):
#         print('i', 'TCruce[i]','TExterior[i]', 'THall[i]', 'TPolicromos[i]')
#         print(i, TCruce[i],TExterior[i], THall[i], TPolicromos[i])
        Tc_menosTe.append(TCruce[i]-TExterior[i])
        Th_menosTe.append(THall[i]-TExterior[i])
        Tp_menosTe.append(TPolicromos[i]-TExterior[i])
#         print(Tc_menosTe, Th_menosTe, Tp_menosTe)
        i=i+1
#     print(len(Tc_menosTe), len(Th_menosTe),len(Tp_menosTe))
#     Tc_menosTe= TCruce-TExterior
#     Th_menosTe= THall-TExterior
#     Tp_menosTe= TPolicromos-TExterior
    #Mediante scipy:
    r_DTcTe_CO2c, p_DTcTe_CO2c = stats.pearsonr(Tc_menosTe, CO2Cruce)
    r_DThTe_CO2h, p_DThTe_CO2h = stats.pearsonr(Th_menosTe, CO2Hall)
    r_DTpTe_CO2p, p_DTpTe_CO2p = stats.pearsonr(Tp_menosTe, CO2Policromos)
    return(r_DTcTe_CO2c, p_DTcTe_CO2c, r_DThTe_CO2h, p_DThTe_CO2h, r_DTpTe_CO2p, p_DTpTe_CO2p)

def CoefPearsonCO2_mesesSinTc(NombreFicheroExterior, NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
    Exterior = pd.read_excel(NombreFicheroExterior)
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce
    CO2Hall = []
    TExterior = []
    THall = []
    while a<len(listasPolicromos[2]):
        CO2Hall.append(Hall['CO2 A (ppm)'][a])
        TExterior.append(Exterior["Tr Exterior (C Tr)"][a])
        THall.append(Hall['Temperatura A (° C)'][a])
        a=a+1
    CO2Policromos = listasPolicromos[2]
    TPolicromos = listasPolicromos[0]
    #Eliminar los nans y infs de las listas
    index=len(TPolicromos)-1
    while index>-1:
        if  sum([pd.isna(CO2Hall[index]), pd.isna(TExterior[index]), 
                 pd.isna(THall[index]), pd.isna(listasPolicromos[0][index]), pd.isna(listasPolicromos[2][index])]) > 0:
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
            TExterior.pop(index)
            THall.pop(index)
            TPolicromos.pop(index)
        index=index-1
    #Diferencia de temeperaturas entre las salas y el exterior:
    #No deja hacer la resta directamente, hacerlo elemento por elemento en un bucle
    i=0
    Th_menosTe= []
    Tp_menosTe= []
#     print(len(TExterior), len(THall),len(TPolicromos))
    while i<len(TExterior):
#         print('i', 'TExterior[i]', 'THall[i]', 'TPolicromos[i]')
#         print(i, TExterior[i], THall[i], TPolicromos[i])
        Th_menosTe.append(THall[i]-TExterior[i])
        Tp_menosTe.append(TPolicromos[i]-TExterior[i])
#         print( Th_menosTe, Tp_menosTe)
        i=i+1
#     print(len(Th_menosTe),len(Tp_menosTe))

    #Mediante scipy:
#     r_DTcTe_CO2c, p_DTcTe_CO2c = stats.pearsonr(Tc_menosTe, CO2Cruce)
    r_DThTe_CO2h, p_DThTe_CO2h = stats.pearsonr(Th_menosTe, CO2Hall)
    r_DTpTe_CO2p, p_DTpTe_CO2p = stats.pearsonr(Tp_menosTe, CO2Policromos)
    return(float('nan'), float('nan'), r_DThTe_CO2h, p_DThTe_CO2h, r_DTpTe_CO2p, p_DTpTe_CO2p)


In [5]:
#2020
r10_2020 = CoefPearsonCO2("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              DatosPolicromos_10_20)     
r11_2020 = CoefPearsonCO2("./altamira/2020/11.NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_20)
r12_2020 = CoefPearsonCO2("./altamira/2020/12.DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_12_20)
#2021
r1_2021 = CoefPearsonCO2("./altamira/2021/1.ENERO/ZONA CRUCE ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA EXTERIOR ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA ENTRADA ENERO.xlsx",
                                                          DatosPolicromos_1_21)
r2_2021 = CoefPearsonCO2("./altamira/2021/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_21)
r3_2021 = CoefPearsonCO2("./altamira/2021/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_21)
r4_2021 = CoefPearsonCO2("./altamira/2021/4.ABRIL/ZONA CRUCE.xlsx",
                                                          "./altamira/2021/4.ABRIL/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/4.ABRIL/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_4_21)
r5_2021 = CoefPearsonCO2("./altamira/2021/5.MAYO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_5_21)
r6_2021 = CoefPearsonCO2("./altamira/2021/6.JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/6.JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/6.JUNIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_6_21)
r7_2021 = CoefPearsonCO2("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_7_21)
r8_2021 = CoefPearsonCO2("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_21)
# r8_2021 = float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan')
r9_2021 = CoefPearsonCO2("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_21)
r10_2021 = CoefPearsonCO2("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_21)
r11_2021 = CoefPearsonCO2("./altamira/2021/11. NOVIEMBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_21)
r12_2021 = CoefPearsonCO2("./altamira/2021/12. DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2021/12. DICIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/12. DICIEMBRE/ZONA ENTRADA.xlsx",
                                                              DatosPolicromos_12_21)
#2022
r1_2022 = CoefPearsonCO2("./altamira/2022/1.ENERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/1.ENERO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/1.ENERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_1_22)
#2,3,4,5 no tienen Tc:
r2_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_22)
r3_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/3.MARZO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_22)
r4_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/4.ABRIL/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_4_22)
r5_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_5_22)

r6_2022 = CoefPearsonCO2("./altamira/2022/6. JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/6. JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/6. JUNIO/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_6_22)
# r7_2022 = CoefPearsonSinCorrec("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx", 
#                                                           "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA POLICROMOS C.xlsx")
r7_2022 = CoefPearsonCO2("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
                                                              "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx", 
                                                              DatosPolicromos_7_22)     
r8_2022 = CoefPearsonCO2("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_22)
r9_2022 = CoefPearsonCO2("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_22)
r10_2022 = CoefPearsonCO2("./altamira/2022/10. OCTUBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_22)
r11_2022 = CoefPearsonCO2("./altamira/2022/11. NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_22)
r12_2022 = CoefPearsonCO2("./altamira/2022/12. DICIEMBRE/ZONA CRUCE D.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                               DatosPolicromos_12_22)


#Ordenar los coef de correl en una tabla con scypy:
#r = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
s1=pd.Series([r10_2020[0],r11_2020[0], r12_2020[0], r1_2021[0], r2_2021[0], r3_2021[0], r4_2021[0], r5_2021[0], r6_2021[0], r7_2021[0], r8_2021[0], r9_2021[0], r10_2021[0], r11_2021[0], r12_2021[0],r1_2022[0], r2_2022[0], r3_2022[0], r4_2022[0], r5_2022[0], r6_2022[0], r7_2022[0], r8_2022[0], r9_2022[0], r10_2022[0], r11_2022[0], r12_2022[0] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s2=pd.Series([r10_2020[1],r11_2020[1], r12_2020[1], r1_2021[1], r2_2021[1], r3_2021[1], r4_2021[1], r5_2021[1], r6_2021[1], r7_2021[1], r8_2021[1], r9_2021[1], r10_2021[1], r11_2021[1], r12_2021[1] ,r1_2022[1], r2_2022[1], r3_2022[1], r4_2022[1], r5_2022[1], r6_2022[1], r7_2022[1], r8_2022[1], r9_2022[1], r10_2022[1], r11_2022[1], r12_2022[1] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s3=pd.Series([r10_2020[2],r11_2020[2], r12_2020[2], r1_2021[2], r2_2021[2], r3_2021[2], r4_2021[2], r5_2021[2], r6_2021[2], r7_2021[2], r8_2021[2], r9_2021[2], r10_2021[2], r11_2021[2], r12_2021[2],r1_2022[2], r2_2022[2], r3_2022[2], r4_2022[2], r5_2022[2], r6_2022[2], r7_2022[2], r8_2022[2], r9_2022[2], r10_2022[2], r11_2022[2], r12_2022[2] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s4=pd.Series([r10_2020[3],r11_2020[3], r12_2020[3], r1_2021[3], r2_2021[3], r3_2021[3], r4_2021[3], r5_2021[3], r6_2021[3], r7_2021[3], r8_2021[3], r9_2021[3], r10_2021[3], r11_2021[3], r12_2021[3],r1_2022[3], r2_2022[3], r3_2022[3], r4_2022[3], r5_2022[3], r6_2022[3], r7_2022[3], r8_2022[3], r9_2022[3], r10_2022[3], r11_2022[3], r12_2022[3] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s5=pd.Series([r10_2020[4],r11_2020[4], r12_2020[4], r1_2021[4], r2_2021[4], r3_2021[4], r4_2021[1], r5_2021[4], r6_2021[4], r7_2021[4], r8_2021[4], r9_2021[4], r10_2021[4], r11_2021[4], r12_2021[4] ,r1_2022[4], r2_2022[4], r3_2022[4], r4_2022[4], r5_2022[4], r6_2022[4], r7_2022[4], r8_2022[4], r9_2022[4], r10_2022[4], r11_2022[4], r12_2022[4] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s6=pd.Series([r10_2020[5],r11_2020[5], r12_2020[5], r1_2021[5], r2_2021[5], r3_2021[5], r4_2021[5], r5_2021[5], r6_2021[5], r7_2021[5], r8_2021[5], r9_2021[5], r10_2021[5], r11_2021[5], r12_2021[5],r1_2022[5], r2_2022[5], r3_2022[5], r4_2022[5], r5_2022[5], r6_2022[5], r7_2022[5], r8_2022[5], r9_2022[5], r10_2022[5], r11_2022[5], r12_2022[5] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['CC_TcmenosTe_CO2c', 'pvalue_TcmenosTe_CO2c', 'CC_ThmenosTe_CO2h',  'pvalue_ThmenosTe_CO2h',
                      'CC_TpmenosTe_CO2p', 'pvalue_TpmenosTe_CO2p']
print(CoefCorrel)

#Sacar las estadísticas de los coef de correl
CoefCorrel.describe()

                 CC_TcmenosTe_CO2c  pvalue_TcmenosTe_CO2c  CC_ThmenosTe_CO2h  \
Octubre 2020              0.337751           1.109163e-60           0.604019   
Noviembre 2020            0.278902           6.986314e-49           0.482714   
Diciembre 2020           -0.213811           4.105330e-32           0.407488   
Enero 2021                0.102894           2.237915e-08           0.584745   
Febrero 2021             -0.212997           5.963960e-29           0.211188   
Marzo 2021                0.445558          4.471468e-145           0.687644   
Abril 2021                0.451990          4.999873e-145           0.614877   
Mayo 2021                 0.198337           1.354079e-27           0.598131   
Junio 2021                0.490534          2.878453e-174           0.516950   
Julio 2021                0.569499          2.046987e-255           0.334373   
Agosto 2021               0.601001          1.838325e-239           0.235398   
Septiembre 2021           0.589058      

,CC_TcmenosTe_CO2c,pvalue_TcmenosTe_CO2c,CC_ThmenosTe_CO2h,pvalue_ThmenosTe_CO2h,CC_TpmenosTe_CO2p,pvalue_TpmenosTe_CO2p
count,23.000000,2.300000e+01,27.000000,2.700000e+01,27.000000,2.700000e+01
mean,0.260252,9.730065e-10,0.433827,2.258813e-02,0.186881,3.088519e-02
std,0.325686,4.666375e-09,0.184546,8.215307e-02,0.246003,8.239367e-02
min,-0.365413,9.529013e-275,-0.021531,0.000000e+00,-0.313174,3.458041e-229
25%,0.127348,2.235734e-145,0.360513,2.396471e-222,0.025847,1.768807e-97
50%,0.374021,2.632995e-96,0.450576,1.863631e-130,0.172506,6.376609e-35
75%,0.495283,3.379801e-38,0.589067,1.505985e-70,0.403519,1.002681e-05
max,0.602390,2.237915e-08,0.687644,3.454170e-01,0.556127,2.942691e-01


In [6]:
r10_2020

(0.3377506026108978,
 1.109162797989957e-60,
 0.6040190471992246,
 4.5899356913470416e-222,
 0.06305900346763058,
 0.0028781570234224893)

# 1.2) ¿Las oscilaciones diarias se deben a un intercambio de aire con el exterior? Coeficientes de correlación entre la diferencia de temperaturas entre exterior y cada sala con la cantidad de CO2. Variables suavizadas

In [7]:
def CoefPearsonCO2(NombreFicheroCruce, NombreFicheroExterior, NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
    Cruce= pd.read_excel(NombreFicheroCruce)
    Exterior = pd.read_excel(NombreFicheroExterior)
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce:
    CO2Cruce = []
    CO2Hall = []
    TCruce = []
    TExterior = []
    THall = []
    while a<len(listasPolicromos[2]):
        CO2Cruce.append(Cruce['CO2c_suavizado'][a])
        CO2Hall.append(Hall['CO2h_suavizado'][a])
        TCruce.append(Cruce['Tc_suavizada'][a])
        TExterior.append(Exterior['Te_suavizada'][a])
        THall.append(Hall['Th_suavizada'][a])
        a=a+1
    CO2Policromos = listasPolicromos[3]
    TPolicromos = listasPolicromos[1]
    #Eliminar los nans y infs de las listas
    index=len(TPolicromos)-1
    while index>-1:
        if  sum([pd.isna(CO2Cruce[index]), pd.isna(CO2Hall[index]), pd.isna(TCruce[index]), pd.isna(TExterior[index]), 
                 pd.isna(THall[index]), pd.isna(listasPolicromos[1][index]), pd.isna(listasPolicromos[3][index])]) > 0:
            CO2Cruce.pop(index)
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
            TCruce.pop(index)
            TExterior.pop(index)
            THall.pop(index)
            TPolicromos.pop(index)
        index=index-1
#     print(len(TsPolicromos), TsExterior)

    #Diferencia de temeperaturas entre las salas y el exterior:
    #No deja hacer la resta directamente, hacerlo elemento por elemento en un bucle
    i=0
    Tc_menosTe= []
    Th_menosTe= []
    Tp_menosTe= []
#     print(len(TCruce), len(TExterior),len(THall),len(TPolicromos))
    while i<len(CO2Cruce):
#         print(i)
        Tc_menosTe.append(TCruce[i]-TExterior[i])
        Th_menosTe.append(THall[i]-TExterior[i])
        Tp_menosTe.append(TPolicromos[i]-TExterior[i])
#         print(Th_menosTe)
        i=i+1
#     print(len(Tc_menosTe), len(Tp_menosTe),len(Th_menosTe),len(CO2Cruce),len(CO2Cruce),len(CO2Cruce))
#     print('Tp_menosTe')
    l=0
    j=0
#     while l< len(Tp_menosTe):
# #         print(Tp_menosTe[l])
#         if float(Tp_menosTe[l])==float('nan'):
#             print(l, Tp_menosTe[l])
#         l=l+1
#     print( 'CO2Policromos')
#     while j< len(Tp_menosTe):
#         if Tp_menosTe[j]==float('nan'):
#             print(j, Tp_menosTe[j])
#         j=j+1
#     Mediante scipy:
    r_DTcTe_CO2c, p_DTcTe_CO2c = stats.pearsonr(Tc_menosTe, CO2Cruce)
    r_DThTe_CO2h, p_DThTe_CO2h = stats.pearsonr(Th_menosTe, CO2Hall)
    r_DTpTe_CO2p, p_DTpTe_CO2p = stats.pearsonr(Tp_menosTe, CO2Policromos)
    return(r_DTcTe_CO2c, p_DTcTe_CO2c, r_DThTe_CO2h, p_DThTe_CO2h, r_DTpTe_CO2p, p_DTpTe_CO2p)
#     return(Tp_menosTe, CO2Policromos)

In [8]:
def CoefPearsonCO2_mesesSinTc(NombreFicheroExterior, NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
    Exterior = pd.read_excel(NombreFicheroExterior)
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce
    CO2Hall = []
    TExterior = []
    THall = []
    while a<len(listasPolicromos[2]):
        CO2Hall.append(Hall['CO2h_suavizado'][a])
        TExterior.append(Exterior['Te_suavizada'][a])
        THall.append(Hall['Th_suavizada'][a])
        a=a+1
    CO2Policromos = listasPolicromos[3]
    TPolicromos = listasPolicromos[1]
    #Eliminar los nans y infs de las listas
    index=len(TPolicromos)-1
    while index>-1:
#         print(index)
        if  sum([pd.isna(CO2Hall[index]), pd.isna(TExterior[index]), 
                 pd.isna(THall[index]), pd.isna(listasPolicromos[0][index]), pd.isna(listasPolicromos[2][index])]) > 0:
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
            TExterior.pop(index)
            THall.pop(index)
            TPolicromos.pop(index)
        index=index-1
    #Diferencia de temeperaturas entre las salas y el exterior:
    #No deja hacer la resta directamente, hacerlo elemento por elemento en un bucle
    i=0
    Th_menosTe= []
    Tp_menosTe= []
#     print(len(TExterior), len(THall),len(TPolicromos))
    while i<len(TExterior):
#         print('i', 'TExterior[i]', 'THall[i]', 'TPolicromos[i]')
#         print(i, TExterior[i], THall[i], TPolicromos[i])
        Th_menosTe.append(THall[i]-TExterior[i])
        Tp_menosTe.append(TPolicromos[i]-TExterior[i])
#         print( Th_menosTe, Tp_menosTe)
        i=i+1
#     print(len(Th_menosTe),len(Tp_menosTe))

    #Mediante scipy:
#     r_DTcTe_CO2c, p_DTcTe_CO2c = stats.pearsonr(Tc_menosTe, CO2Cruce)
    r_DThTe_CO2h, p_DThTe_CO2h = stats.pearsonr(Th_menosTe, CO2Hall)
    r_DTpTe_CO2p, p_DTpTe_CO2p = stats.pearsonr(Tp_menosTe, CO2Policromos)
    return(float('nan'), float('nan'), r_DThTe_CO2h, p_DThTe_CO2h, r_DTpTe_CO2p, p_DTpTe_CO2p)

In [11]:
#2020
r10_2020 = CoefPearsonCO2("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              DatosPolicromos_10_20)     
r11_2020 = CoefPearsonCO2("./altamira/2020/11.NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_20)
r12_2020 = CoefPearsonCO2("./altamira/2020/12.DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_12_20)
#2021
r1_2021 = CoefPearsonCO2("./altamira/2021/1.ENERO/ZONA CRUCE ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA EXTERIOR ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA ENTRADA ENERO.xlsx",
                                                          DatosPolicromos_1_21)
r2_2021 = CoefPearsonCO2("./altamira/2021/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_21)
r3_2021 = CoefPearsonCO2("./altamira/2021/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_21)
r4_2021 = CoefPearsonCO2("./altamira/2021/4.ABRIL/ZONA CRUCE.xlsx",
                                                          "./altamira/2021/4.ABRIL/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/4.ABRIL/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_4_21)
r5_2021 = CoefPearsonCO2("./altamira/2021/5.MAYO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_5_21)
r6_2021 = CoefPearsonCO2("./altamira/2021/6.JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/6.JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/6.JUNIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_6_21)
r7_2021 = CoefPearsonCO2("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/7.JULIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_7_21)
r8_2021 = CoefPearsonCO2("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/8.AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_21)
# r8_2021 = float('nan'), float('nan'), float('nan'), float('nan'), float('nan'), float('nan')
r9_2021 = CoefPearsonCO2("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_21)
r10_2021 = CoefPearsonCO2("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_21)
r11_2021 = CoefPearsonCO2("./altamira/2021/11. NOVIEMBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_21)
r12_2021 = CoefPearsonCO2("./altamira/2021/12. DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2021/12. DICIEMBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2021/12. DICIEMBRE/ZONA ENTRADA.xlsx",
                                                              DatosPolicromos_12_21)
#2022
r1_2022 = CoefPearsonCO2("./altamira/2022/1.ENERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/1.ENERO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/1.ENERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_1_22)
#2,3,4,5 no tienen Tc:
r2_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/2.FEBRERO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_22)
r3_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/3.MARZO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_22)
r4_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/4.ABRIL/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_4_22)
r5_2022 = CoefPearsonCO2_mesesSinTc("./altamira/2022/5.MAYO/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/5.MAYO/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_5_22)

r6_2022 = CoefPearsonCO2("./altamira/2022/6. JUNIO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/6. JUNIO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/6. JUNIO/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_6_22)
# r7_2022 = CoefPearsonSinCorrec("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx", 
#                                                           "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA POLICROMOS C.xlsx")
r7_2022 = CoefPearsonCO2("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
                                                              "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx", 
                                                              DatosPolicromos_7_22)     
r8_2022 = CoefPearsonCO2("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/8. AGOSTO/ZONA EXTERIOR.xlsx", 
                                                          "./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_22)
r9_2022 = CoefPearsonCO2("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA EXTERIOR.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_22)
r10_2022 = CoefPearsonCO2("./altamira/2022/10. OCTUBRE/ZONA CRUCE D.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA EXTERIOR.xlsx", 
                                                              "./altamira/2022/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_22)
r11_2022 = CoefPearsonCO2("./altamira/2022/11. NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_22)
r12_2022 = CoefPearsonCO2("./altamira/2022/12. DICIEMBRE/ZONA CRUCE D.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA EXTERIOR.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                               DatosPolicromos_12_22)


#Ordenar los coef de correl en una tabla con scypy:
#r = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
s1=pd.Series([r10_2020[0],r11_2020[0], r12_2020[0], r1_2021[0], r2_2021[0], r3_2021[0], r4_2021[0], r5_2021[0], r6_2021[0], r7_2021[0], r8_2021[0], r9_2021[0], r10_2021[0], r11_2021[0], r12_2021[0],r1_2022[0], r2_2022[0], r3_2022[0], r4_2022[0], r5_2022[0], r6_2022[0], r7_2022[0], r8_2022[0], r9_2022[0], r10_2022[0], r11_2022[0], r12_2022[0] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s2=pd.Series([r10_2020[1],r11_2020[1], r12_2020[1], r1_2021[1], r2_2021[1], r3_2021[1], r4_2021[1], r5_2021[1], r6_2021[1], r7_2021[1], r8_2021[1], r9_2021[1], r10_2021[1], r11_2021[1], r12_2021[1] ,r1_2022[1], r2_2022[1], r3_2022[1], r4_2022[1], r5_2022[1], r6_2022[1], r7_2022[1], r8_2022[1], r9_2022[1], r10_2022[1], r11_2022[1], r12_2022[1] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s3=pd.Series([r10_2020[2],r11_2020[2], r12_2020[2], r1_2021[2], r2_2021[2], r3_2021[2], r4_2021[2], r5_2021[2], r6_2021[2], r7_2021[2], r8_2021[2], r9_2021[2], r10_2021[2], r11_2021[2], r12_2021[2],r1_2022[2], r2_2022[2], r3_2022[2], r4_2022[2], r5_2022[2], r6_2022[2], r7_2022[2], r8_2022[2], r9_2022[2], r10_2022[2], r11_2022[2], r12_2022[2] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s4=pd.Series([r10_2020[3],r11_2020[3], r12_2020[3], r1_2021[3], r2_2021[3], r3_2021[3], r4_2021[3], r5_2021[3], r6_2021[3], r7_2021[3], r8_2021[3], r9_2021[3], r10_2021[3], r11_2021[3], r12_2021[3],r1_2022[3], r2_2022[3], r3_2022[3], r4_2022[3], r5_2022[3], r6_2022[3], r7_2022[3], r8_2022[3], r9_2022[3], r10_2022[3], r11_2022[3], r12_2022[3] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s5=pd.Series([r10_2020[4],r11_2020[4], r12_2020[4], r1_2021[4], r2_2021[4], r3_2021[4], r4_2021[1], r5_2021[4], r6_2021[4], r7_2021[4], r8_2021[4], r9_2021[4], r10_2021[4], r11_2021[4], r12_2021[4] ,r1_2022[4], r2_2022[4], r3_2022[4], r4_2022[4], r5_2022[4], r6_2022[4], r7_2022[4], r8_2022[4], r9_2022[4], r10_2022[4], r11_2022[4], r12_2022[4] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s6=pd.Series([r10_2020[5],r11_2020[5], r12_2020[5], r1_2021[5], r2_2021[5], r3_2021[5], r4_2021[5], r5_2021[5], r6_2021[5], r7_2021[5], r8_2021[5], r9_2021[5], r10_2021[5], r11_2021[5], r12_2021[5],r1_2022[5], r2_2022[5], r3_2022[5], r4_2022[5], r5_2022[5], r6_2022[5], r7_2022[5], r8_2022[5], r9_2022[5], r10_2022[5], r11_2022[5], r12_2022[5] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['CC_TcmenosTe_CO2c', 'pvalue_TcmenosTe_CO2c', 'CC_ThmenosTe_CO2h',  'pvalue_ThmenosTe_CO2h',
                      'CC_TpmenosTe_CO2p', 'pvalue_TpmenosTe_CO2p']
print(CoefCorrel)

#Sacar las estadísticas de los coef de correl
CoefCorrel.describe()

                 CC_TcmenosTe_CO2c  pvalue_TcmenosTe_CO2c  CC_ThmenosTe_CO2h  \
Octubre 2020              0.339892           2.410671e-61           0.607858   
Noviembre 2020            0.278828           1.069560e-48           0.484397   
Diciembre 2020           -0.213936           4.249422e-32           0.411007   
Enero 2021                0.099752           6.409061e-08           0.588404   
Febrero 2021             -0.213365           5.370414e-29           0.212490   
Marzo 2021                0.444810          4.647401e-144           0.690368   
Abril 2021                0.453276          1.078006e-145           0.617060   
Mayo 2021                 0.200954           3.298991e-28           0.604290   
Junio 2021                0.495512          5.110880e-178           0.522720   
Julio 2021                0.572557          5.501525e-258           0.337545   
Agosto 2021               0.604813          8.420836e-242           0.240453   
Septiembre 2021           0.594158      

,CC_TcmenosTe_CO2c,pvalue_TcmenosTe_CO2c,CC_ThmenosTe_CO2h,pvalue_ThmenosTe_CO2h,CC_TpmenosTe_CO2p,pvalue_TpmenosTe_CO2p
count,23.000000,2.300000e+01,27.000000,2.700000e+01,27.000000,2.700000e+01
mean,0.261611,2.786548e-09,0.437459,2.502808e-02,0.189186,2.560569e-02
std,0.328393,1.336382e-08,0.185598,9.138775e-02,0.251679,8.283789e-02
min,-0.369885,7.937892e-277,-0.020677,0.000000e+00,-0.326338,9.091021e-227
25%,0.125257,5.390030e-146,0.366417,1.992801e-225,0.023159,5.170374e-105
50%,0.378086,2.280839e-98,0.456721,4.423365e-131,0.190207,9.435552e-40
75%,0.499637,6.156441e-40,0.593511,1.186235e-72,0.418714,4.140167e-10
max,0.604976,6.409061e-08,0.690368,3.914195e-01,0.567169,4.086857e-01


In [ ]:
r10_2020 = CoefPearsonCO2("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA EXTERIOR.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              DatosPolicromos_10_20)

In [ ]:
Cruce=pd.read_excel("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx")
Cruce

# 2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables sin suavizar

In [13]:
def CoefPearsonCO2entreSalas(NombreFicheroCruce, NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
    Cruce= pd.read_excel(NombreFicheroCruce)
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce:
    CO2Cruce = []
    CO2Hall = []
    while a<len(listasPolicromos[2]):
        CO2Cruce.append(Cruce['CO2 D (ppm)'][a])
        CO2Hall.append(Hall['CO2 A (ppm)'][a])
        a=a+1
    CO2Policromos = listasPolicromos[2]
    #Eliminar los nans y infs de las listas
    index=len(CO2Policromos)-1
    while index>-1:
        if  sum([pd.isna(CO2Cruce[index]), pd.isna(CO2Hall[index]), pd.isna(CO2Policromos[index])]) > 0:
            CO2Cruce.pop(index)
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
        index=index-1
    #Mediante scipy:
    r_CO2ch, p_CO2ch = stats.pearsonr(CO2Cruce, CO2Hall)
    r_CO2hp, p_CO2hp = stats.pearsonr(CO2Hall, CO2Policromos)
    r_CO2pc, p_CO2pc = stats.pearsonr(CO2Cruce, CO2Policromos)
    return(r_CO2ch, p_CO2ch, r_CO2hp, p_CO2hp, r_CO2pc, p_CO2pc)

In [14]:
def CCoefPearsonCO2entreSalasmayo22(NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
#     Mayo22 no tiene archivo cruce
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce:
    CO2Hall = []
    while a<len(listasPolicromos[3]):
        CO2Hall.append(Hall['CO2 A (ppm)'][a])
        a=a+1
    CO2Policromos = listasPolicromos[2]
    #Eliminar los nans y infs de las listas
    index=len(CO2Policromos)-1
    while index>-1:
        if  sum([pd.isna(CO2Hall[index]), pd.isna(CO2Policromos[index])]) > 0:
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
        index=index-1
    #Mediante scipy:
#     r_CO2ch, p_CO2ch = stats.pearsonr(CO2Cruce, CO2Hall)
    r_CO2hp, p_CO2hp = stats.pearsonr(CO2Hall, CO2Policromos)
#     r_CO2pc, p_CO2pc = stats.pearsonr(CO2Cruce, CO2Policromos)
    return(float('nan'), float('nan'), r_CO2hp, p_CO2hp,float('nan'), float('nan'))

In [20]:
tiempoInicial= time.time()
#2020
r10_2020 = CoefPearsonCO2entreSalas("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              DatosPolicromos_10_20)     
r11_2020 = CoefPearsonCO2entreSalas("./altamira/2020/11.NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_20)
r12_2020 = CoefPearsonCO2entreSalas("./altamira/2020/12.DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_12_20)
#2021
r1_2021 = CoefPearsonCO2entreSalas("./altamira/2021/1.ENERO/ZONA CRUCE ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA ENTRADA ENERO.xlsx",
                                                          DatosPolicromos_1_21)
r2_2021 = CoefPearsonCO2entreSalas("./altamira/2021/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_21)
r3_2021 = CoefPearsonCO2entreSalas("./altamira/2021/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_21)
r4_2021 = CoefPearsonCO2entreSalas("./altamira/2021/4.ABRIL/ZONA CRUCE.xlsx", 
                                                          "./altamira/2021/4.ABRIL/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_4_21)
r5_2021 = CoefPearsonCO2entreSalas("./altamira/2021/5.MAYO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_5_21)
r6_2021 = CoefPearsonCO2entreSalas("./altamira/2021/6.JUNIO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2021/6.JUNIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_6_21)
r7_2021 = CoefPearsonCO2entreSalas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_7_21)
r8_2021 = CoefPearsonCO2entreSalas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_21)
r9_2021 = CoefPearsonCO2entreSalas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_21)
r10_2021 = CoefPearsonCO2entreSalas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_21)
r11_2021 = CoefPearsonCO2entreSalas("./altamira/2021/11. NOVIEMBRE/ZONA CRUCE D.xlsx",  
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_21)
r12_2021 = CoefPearsonCO2entreSalas("./altamira/2021/12. DICIEMBRE/ZONA CRUCE.xlsx", 
                                                              "./altamira/2021/12. DICIEMBRE/ZONA ENTRADA.xlsx",
                                                              DatosPolicromos_12_21)
#2022
r1_2022 = CoefPearsonCO2entreSalas("./altamira/2022/1.ENERO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/1.ENERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_1_22)
#2,3,4,5 no tienen Tc:
r2_2022 = CoefPearsonCO2entreSalas("./altamira/2022/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_22)
r3_2022 = CoefPearsonCO2entreSalas("./altamira/2022/3.MARZO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_22)
r4_2022 = CoefPearsonCO2entreSalas("./altamira/2022/4.ABRIL/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_4_22)
r5_2022 = CCoefPearsonCO2entreSalasmayo22("./altamira/2022/5.MAYO/ZONA ENTRADA.xlsx", DatosPolicromos_5_22)
#no hay cruce de mayo


r6_2022 = CoefPearsonCO2entreSalas("./altamira/2022/6. JUNIO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/6. JUNIO/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_6_22)
# r7_2022 = CoefPearsonSinCorrec("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx", 
#                                                           "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA POLICROMOS C.xlsx")
r7_2022 = CoefPearsonCO2entreSalas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
                                                              "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx", 
                                                              DatosPolicromos_7_22)     
r8_2022 = CoefPearsonCO2entreSalas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_22)
r9_2022 = CoefPearsonCO2entreSalas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_22)
r10_2022 = CoefPearsonCO2entreSalas("./altamira/2022/10. OCTUBRE/ZONA CRUCE D.xlsx",  
                                                              "./altamira/2022/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_22)
r11_2022 = CoefPearsonCO2entreSalas("./altamira/2022/11. NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_22)
r12_2022 = CoefPearsonCO2entreSalas("./altamira/2022/12. DICIEMBRE/ZONA CRUCE D.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                               DatosPolicromos_12_22)


#Ordenar los coef de correl en una tabla con scypy:
#r = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
s1=pd.Series([r10_2020[0],r11_2020[0], r12_2020[0], r1_2021[0], r2_2021[0], r3_2021[0], r4_2021[0], r5_2021[0], r6_2021[0], r7_2021[0], r8_2021[0], r9_2021[0], r10_2021[0], r11_2021[0], r12_2021[0],r1_2022[0], r2_2022[0], r3_2022[0], r4_2022[0], r5_2022[0], r6_2022[0], r7_2022[0], r8_2022[0], r9_2022[0], r10_2022[0], r11_2022[0], r12_2022[0] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s2=pd.Series([r10_2020[1],r11_2020[1], r12_2020[1], r1_2021[1], r2_2021[1], r3_2021[1], r4_2021[1], r5_2021[1], r6_2021[1], r7_2021[1], r8_2021[1], r9_2021[1], r10_2021[1], r11_2021[1], r12_2021[1] ,r1_2022[1], r2_2022[1], r3_2022[1], r4_2022[1], r5_2022[1], r6_2022[1], r7_2022[1], r8_2022[1], r9_2022[1], r10_2022[1], r11_2022[1], r12_2022[1] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s3=pd.Series([r10_2020[2],r11_2020[2], r12_2020[2], r1_2021[2], r2_2021[2], r3_2021[2], r4_2021[2], r5_2021[2], r6_2021[2], r7_2021[2], r8_2021[2], r9_2021[2], r10_2021[2], r11_2021[2], r12_2021[2],r1_2022[2], r2_2022[2], r3_2022[2], r4_2022[2], r5_2022[2], r6_2022[2], r7_2022[2], r8_2022[2], r9_2022[2], r10_2022[2], r11_2022[2], r12_2022[2] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s4=pd.Series([r10_2020[3],r11_2020[3], r12_2020[3], r1_2021[3], r2_2021[3], r3_2021[3], r4_2021[3], r5_2021[3], r6_2021[3], r7_2021[3], r8_2021[3], r9_2021[3], r10_2021[3], r11_2021[3], r12_2021[3],r1_2022[3], r2_2022[3], r3_2022[3], r4_2022[3], r5_2022[3], r6_2022[3], r7_2022[3], r8_2022[3], r9_2022[3], r10_2022[3], r11_2022[3], r12_2022[3] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s5=pd.Series([r10_2020[4],r11_2020[4], r12_2020[4], r1_2021[4], r2_2021[4], r3_2021[4], r4_2021[1], r5_2021[4], r6_2021[4], r7_2021[4], r8_2021[4], r9_2021[4], r10_2021[4], r11_2021[4], r12_2021[4] ,r1_2022[4], r2_2022[4], r3_2022[4], r4_2022[4], r5_2022[4], r6_2022[4], r7_2022[4], r8_2022[4], r9_2022[4], r10_2022[4], r11_2022[4], r12_2022[4] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s6=pd.Series([r10_2020[5],r11_2020[5], r12_2020[5], r1_2021[5], r2_2021[5], r3_2021[5], r4_2021[5], r5_2021[5], r6_2021[5], r7_2021[5], r8_2021[5], r9_2021[5], r10_2021[5], r11_2021[5], r12_2021[5],r1_2022[5], r2_2022[5], r3_2022[5], r4_2022[5], r5_2022[5], r6_2022[5], r7_2022[5], r8_2022[5], r9_2022[5], r10_2022[5], r11_2022[5], r12_2022[5] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['r_CO2ch', 'p_CO2ch', 'r_CO2hp', 'p_CO2hp', 'r_CO2pc', 'p_CO2pc']
print(CoefCorrel)

#Sacar las estadísticas de los coef de correl
CoefCorrel.describe()
tiempoFinal = time.time()
print(tiempoFinal-tiempoInicial)

                  r_CO2ch        p_CO2ch   r_CO2hp        p_CO2hp   r_CO2pc  \
Octubre 2020     0.868710   0.000000e+00  0.361167   9.703625e-70  0.518232   
Noviembre 2020   0.720465   0.000000e+00  0.446906  3.219528e-131  0.687702   
Diciembre 2020   0.065413   3.558840e-04 -0.087913   1.561771e-06  0.556284   
Enero 2021       0.671337   0.000000e+00  0.564103  6.272683e-247  0.411362   
Febrero 2021     0.637600  1.131407e-306  0.730064   0.000000e+00  0.952484   
Marzo 2021       0.833865   0.000000e+00  0.859149   0.000000e+00  0.911260   
Abril 2021       0.788745   0.000000e+00  0.803174   0.000000e+00  0.000000   
Mayo 2021        0.649327   0.000000e+00  0.613900  1.176042e-305  0.881990   
Junio 2021       0.804915   0.000000e+00  0.746882   0.000000e+00  0.977774   
Julio 2021       0.802693   0.000000e+00  0.719234   0.000000e+00  0.844218   
Agosto 2021      0.770122   0.000000e+00  0.600110  1.895793e-239  0.730769   
Septiembre 2021  0.938172   0.000000e+00  0.857711  

# 2.2) ¿Hay relación entre la cantidad de CO2 que hay en cada sala? Coef correl CO2c-CO2h y CO2c-CO2p con las variables suavizadas

In [22]:
def CoefPearsonCO2entreSalas(NombreFicheroCruce, NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
    Cruce= pd.read_excel(NombreFicheroCruce)
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce:
    CO2Cruce = []
    CO2Hall = []
    while a<len(listasPolicromos[3]):
        CO2Cruce.append(Cruce['CO2c_suavizado'][a])
        CO2Hall.append(Hall['CO2h_suavizado'][a])
        a=a+1
    CO2Policromos = listasPolicromos[3]
    #Eliminar los nans y infs de las listas
    index=len(CO2Policromos)-1
    while index>-1:
        if  sum([pd.isna(CO2Cruce[index]), pd.isna(CO2Hall[index]), pd.isna(CO2Policromos[index])]) > 0:
            CO2Cruce.pop(index)
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
        index=index-1
    #Mediante scipy:
    r_CO2ch, p_CO2ch = stats.pearsonr(CO2Cruce, CO2Hall)
    r_CO2hp, p_CO2hp = stats.pearsonr(CO2Hall, CO2Policromos)
    r_CO2pc, p_CO2pc = stats.pearsonr(CO2Cruce, CO2Policromos)
    return(r_CO2ch, p_CO2ch, r_CO2hp, p_CO2hp, r_CO2pc, p_CO2pc)

In [23]:
def CCoefPearsonCO2entreSalasmayo22(NombreFicheroHall, listasPolicromos):#No hace falta corregir dif temporales, resuelto arriba
#     listasPolicromos = Tp, Tp_suavizada, CO2, CO2_suavizado, Fecha/Tiempo
#     Mayo22 no tiene archivo cruce
    Hall = pd.read_excel(NombreFicheroHall)
    a=0
    #listas de datos a usar de hall, exterior y cruce:
    CO2Hall = []
    while a<len(listasPolicromos[3]):
        CO2Hall.append(Hall['CO2h_suavizado'][a])
        a=a+1
    CO2Policromos = listasPolicromos[3]
    #Eliminar los nans y infs de las listas
    index=len(CO2Policromos)-1
    while index>-1:
        if  sum([pd.isna(CO2Hall[index]), pd.isna(CO2Policromos[index])]) > 0:
            CO2Hall.pop(index)
            CO2Policromos.pop(index)
        index=index-1
    #Mediante scipy:
#     r_CO2ch, p_CO2ch = stats.pearsonr(CO2Cruce, CO2Hall)
    r_CO2hp, p_CO2hp = stats.pearsonr(CO2Hall, CO2Policromos)
#     r_CO2pc, p_CO2pc = stats.pearsonr(CO2Cruce, CO2Policromos)
    return(float('nan'), float('nan'), r_CO2hp, p_CO2hp,float('nan'), float('nan'))

In [24]:
tiempoInicial
#2020
r10_2020 = CoefPearsonCO2entreSalas("./altamira/2020/10.OCTUBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/10.OCTUBRE/ZONA ENTRADA.xlsx", 
                                                              DatosPolicromos_10_20)     
r11_2020 = CoefPearsonCO2entreSalas("./altamira/2020/11.NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2020/11.NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_20)
r12_2020 = CoefPearsonCO2entreSalas("./altamira/2020/12.DICIEMBRE/ZONA CRUCE.xlsx",
                                                              "./altamira/2020/12.DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_12_20)
#2021
r1_2021 = CoefPearsonCO2entreSalas("./altamira/2021/1.ENERO/ZONA CRUCE ENERO.xlsx",
                                                          "./altamira/2021/1.ENERO/ZONA ENTRADA ENERO.xlsx",
                                                          DatosPolicromos_1_21)
r2_2021 = CoefPearsonCO2entreSalas("./altamira/2021/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_21)
r3_2021 = CoefPearsonCO2entreSalas("./altamira/2021/3.MARZO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_21)
r4_2021 = CoefPearsonCO2entreSalas("./altamira/2021/4.ABRIL/ZONA CRUCE.xlsx", 
                                                          "./altamira/2021/4.ABRIL/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_4_21)
r5_2021 = CoefPearsonCO2entreSalas("./altamira/2021/5.MAYO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/5.MAYO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_5_21)
r6_2021 = CoefPearsonCO2entreSalas("./altamira/2021/6.JUNIO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2021/6.JUNIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_6_21)
r7_2021 = CoefPearsonCO2entreSalas("./altamira/2021/7.JULIO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2021/7.JULIO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_7_21)
r8_2021 = CoefPearsonCO2entreSalas("./altamira/2021/8.AGOSTO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2021/8.AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_21)
r9_2021 = CoefPearsonCO2entreSalas("./altamira/2021/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2021/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_21)
r10_2021 = CoefPearsonCO2entreSalas("./altamira/2021/10. OCTUBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2021/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_21)
r11_2021 = CoefPearsonCO2entreSalas("./altamira/2021/11. NOVIEMBRE/ZONA CRUCE D.xlsx",  
                                                              "./altamira/2021/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_21)
r12_2021 = CoefPearsonCO2entreSalas("./altamira/2021/12. DICIEMBRE/ZONA CRUCE.xlsx", 
                                                              "./altamira/2021/12. DICIEMBRE/ZONA ENTRADA.xlsx",
                                                              DatosPolicromos_12_21)
#2022
r1_2022 = CoefPearsonCO2entreSalas("./altamira/2022/1.ENERO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/1.ENERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_1_22)
#2,3,4,5 no tienen Tc:
r2_2022 = CoefPearsonCO2entreSalas("./altamira/2022/2.FEBRERO/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/2.FEBRERO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_2_22)
r3_2022 = CoefPearsonCO2entreSalas("./altamira/2022/3.MARZO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/3.MARZO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_3_22)
r4_2022 = CoefPearsonCO2entreSalas("./altamira/2022/4.ABRIL/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/4.ABRIL/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_4_22)
r5_2022 = CCoefPearsonCO2entreSalasmayo22("./altamira/2022/5.MAYO/ZONA ENTRADA.xlsx", DatosPolicromos_5_22)
#no hay cruce de mayo


r6_2022 = CoefPearsonCO2entreSalas("./altamira/2022/6. JUNIO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/6. JUNIO/ZONA ENTRADA.xlsx",
                                                          DatosPolicromos_6_22)
# r7_2022 = CoefPearsonSinCorrec("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA EXTERIOR.xlsx", 
#                                                           "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx",
#                                                           "./altamira/2022/7.JULIO/ZONA POLICROMOS C.xlsx")
r7_2022 = CoefPearsonCO2entreSalas("./altamira/2022/7.JULIO/ZONA CRUCE C.xlsx",
                                                              "./altamira/2022/7.JULIO/ZONA ENTRADA A.xlsx", 
                                                              DatosPolicromos_7_22)     
r8_2022 = CoefPearsonCO2entreSalas("./altamira/2022/8. AGOSTO/ZONA CRUCE D.xlsx", 
                                                          "./altamira/2022/8. AGOSTO/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_8_22)
r9_2022 = CoefPearsonCO2entreSalas("./altamira/2022/9. SEPTIEMBRE/ZONA CRUCE D.xlsx",
                                                          "./altamira/2022/9. SEPTIEMBRE/ZONA ENTRADA A.xlsx",
                                                          DatosPolicromos_9_22)
r10_2022 = CoefPearsonCO2entreSalas("./altamira/2022/10. OCTUBRE/ZONA CRUCE D.xlsx",  
                                                              "./altamira/2022/10. OCTUBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_10_22)
r11_2022 = CoefPearsonCO2entreSalas("./altamira/2022/11. NOVIEMBRE/ZONA CRUCE D.xlsx",
                                                              "./altamira/2022/11. NOVIEMBRE/ZONA ENTRADA A.xlsx",
                                                              DatosPolicromos_11_22)
r12_2022 = CoefPearsonCO2entreSalas("./altamira/2022/12. DICIEMBRE/ZONA CRUCE D.xlsx",
                                                               "./altamira/2022/12. DICIEMBRE/ZONA ENTRADA A.xlsx",
                                                               DatosPolicromos_12_22)


#Ordenar los coef de correl en una tabla con scypy:
#r = r_TD_Tr, p_TD_Tr, r_TD_Te, p_TD_Te, r_Tr_Te, p_Tr_Te
s1=pd.Series([r10_2020[0],r11_2020[0], r12_2020[0], r1_2021[0], r2_2021[0], r3_2021[0], r4_2021[0], r5_2021[0], r6_2021[0], r7_2021[0], r8_2021[0], r9_2021[0], r10_2021[0], r11_2021[0], r12_2021[0],r1_2022[0], r2_2022[0], r3_2022[0], r4_2022[0], r5_2022[0], r6_2022[0], r7_2022[0], r8_2022[0], r9_2022[0], r10_2022[0], r11_2022[0], r12_2022[0] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s2=pd.Series([r10_2020[1],r11_2020[1], r12_2020[1], r1_2021[1], r2_2021[1], r3_2021[1], r4_2021[1], r5_2021[1], r6_2021[1], r7_2021[1], r8_2021[1], r9_2021[1], r10_2021[1], r11_2021[1], r12_2021[1] ,r1_2022[1], r2_2022[1], r3_2022[1], r4_2022[1], r5_2022[1], r6_2022[1], r7_2022[1], r8_2022[1], r9_2022[1], r10_2022[1], r11_2022[1], r12_2022[1] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s3=pd.Series([r10_2020[2],r11_2020[2], r12_2020[2], r1_2021[2], r2_2021[2], r3_2021[2], r4_2021[2], r5_2021[2], r6_2021[2], r7_2021[2], r8_2021[2], r9_2021[2], r10_2021[2], r11_2021[2], r12_2021[2],r1_2022[2], r2_2022[2], r3_2022[2], r4_2022[2], r5_2022[2], r6_2022[2], r7_2022[2], r8_2022[2], r9_2022[2], r10_2022[2], r11_2022[2], r12_2022[2] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s4=pd.Series([r10_2020[3],r11_2020[3], r12_2020[3], r1_2021[3], r2_2021[3], r3_2021[3], r4_2021[3], r5_2021[3], r6_2021[3], r7_2021[3], r8_2021[3], r9_2021[3], r10_2021[3], r11_2021[3], r12_2021[3],r1_2022[3], r2_2022[3], r3_2022[3], r4_2022[3], r5_2022[3], r6_2022[3], r7_2022[3], r8_2022[3], r9_2022[3], r10_2022[3], r11_2022[3], r12_2022[3] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s5=pd.Series([r10_2020[4],r11_2020[4], r12_2020[4], r1_2021[4], r2_2021[4], r3_2021[4], r4_2021[1], r5_2021[4], r6_2021[4], r7_2021[4], r8_2021[4], r9_2021[4], r10_2021[4], r11_2021[4], r12_2021[4] ,r1_2022[4], r2_2022[4], r3_2022[4], r4_2022[4], r5_2022[4], r6_2022[4], r7_2022[4], r8_2022[4], r9_2022[4], r10_2022[4], r11_2022[4], r12_2022[4] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                  'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
s6=pd.Series([r10_2020[5],r11_2020[5], r12_2020[5], r1_2021[5], r2_2021[5], r3_2021[5], r4_2021[5], r5_2021[5], r6_2021[5], r7_2021[5], r8_2021[5], r9_2021[5], r10_2021[5], r11_2021[5], r12_2021[5],r1_2022[5], r2_2022[5], r3_2022[5], r4_2022[5], r5_2022[5], r6_2022[5], r7_2022[5], r8_2022[5], r9_2022[5], r10_2022[5], r11_2022[5], r12_2022[5] ],
          index = ['Octubre 2020', 'Noviembre 2020', 'Diciembre 2020',
                   'Enero 2021', 'Febrero 2021', 'Marzo 2021', 'Abril 2021', 'Mayo 2021', 'Junio 2021', 'Julio 2021', 'Agosto 2021', 'Septiembre 2021', 'Octubre 2021', 'Noviembre 2021', 'Diciembre 2021',
                   'Enero 2022', 'Febrero 2022', 'Marzo 2022', 'Abril 2022', 'Mayo 2022', 'Junio 2022', 'Julio 2022', 'Agosto 2022', 'Septiembre 2022', 'Octubre 2022', 'Noviembre 2022', 'Diciembre 2022'])
#unir en dataframe
CoefCorrel = pd.concat([s1, s2, s3, s4, s5, s6], axis = 1)
CoefCorrel.columns = ['r_CO2ch', 'p_CO2ch', 'r_CO2hp', 'p_CO2hp', 'r_CO2pc', 'p_CO2pc']
print(CoefCorrel)

#Sacar las estadísticas de los coef de correl
CoefCorrel.describe()
tiempoFinal = time.time()
print(tiempoFinal-tiempoInicial)

                  r_CO2ch        p_CO2ch   r_CO2hp        p_CO2hp   r_CO2pc  \
Octubre 2020     0.870349   0.000000e+00  0.354770   4.840466e-67  0.512678   
Noviembre 2020   0.722409   0.000000e+00  0.460935  4.686048e-140  0.708033   
Diciembre 2020   0.064645   4.222276e-04 -0.081146   9.491812e-06  0.573094   
Enero 2021       0.670721   0.000000e+00  0.561245  7.473984e-243  0.403047   
Febrero 2021     0.644709  4.019386e-315  0.729219   0.000000e+00  0.966647   
Marzo 2021       0.833069   0.000000e+00  0.865823   0.000000e+00  0.924290   
Abril 2021       0.789665   0.000000e+00  0.813150   0.000000e+00  0.000000   
Mayo 2021        0.650675   0.000000e+00  0.625344  2.387325e-319  0.895270   
Junio 2021       0.806298   0.000000e+00  0.758824   0.000000e+00  0.982827   
Julio 2021       0.807926   0.000000e+00  0.743969   0.000000e+00  0.867685   
Agosto 2021      0.774977   0.000000e+00  0.632512  6.300537e-273  0.768784   
Septiembre 2021  0.941257   0.000000e+00  0.869516  